In [5]:
import os
os.getcwd()

'/Users/austinclime/vs_code_projects/vix_approx/vix_approx_research/white_paper_replicate'

In [2]:
from datetime import timedelta
import pytz
import datetime

import pandas as pd
import numpy as np
from numpy.linalg import inv


import matplotlib.pyplot as plt
import timeit

# **Introduction**

### The VIX originates from trying to price the log contract, which pays off log($\frac{S(T)}{S(0)}$) at time T. The log contract gives us insight about expected future volatility.

### The value of the log contract is: $E^*[e^{-rT} log(\frac{S(T)}{S(0)})] = e^{-rT}E^*[rT - \frac{1}{2} \int_{0}^{T} \sigma(t)^2 \,dt]$

### The objective of the VIX is to replicate the payoff of the log contract using a portfolio of out-of-the money options on the S&P 500 Index.

### Let $ \hat{\sigma}^2 = \frac{1}{T} E^*[\int_{0}^{T} \sigma(t)^2 \,dt ]$ and $F_{0,T} = e^{rT}S(0)$

### Then $ \hat{\sigma}^2 = \frac{2e^{rT}}{T} (\int_{0}^{F_{0,T}} \frac{1}{K^2} P(K) \,dK + \int_{F_{0,T}}^{\infty} \frac{1}{K^2} C(K) \,dK)$

### where P(K) is a put with strike price K and C(K) is a call strike price K

### We can think about the VIX as a portfolio of options that approximates the following: $\hat{\sigma} = \frac{2e^{rT}}{T} (\,\sum_{K_i \leq K_0}{\frac{\Delta K_i}{K_i^2}P(K_i)} +  \sum_{K_i > K_0}{\frac{\Delta K_i}{K_i^2}C(K_i)} \,)-\frac{1}{T}(\frac{e^{rT}S(0)}{K_0} - 1)^2$

### where $K_0$ is the first strike price below $e^{rT}S(0), \Delta K_i = \frac{K_{i} - K_{i-1}}{2}$. The last term is a correction term for the fact that there may not be an option with strike price equal to $e^{rT}S(0)$. It is the interval between strike prices - half the difference between the strike on either side of $K_i$.

### (**Note**: $\Delta$K for the lowest strike is simply the difference between the lowest strike and the next higher strike. Likewise, $\Delta$K for the highest strike is the difference between the highest strike and the next lower strike.)

### Q($K_i$) is the midpoint of the bid-ask spread for each option with strike $K_i$

### The framework below for pricing the VIX is adapted from the CBOE's 2009 White Paper. I will mimic the paper's computations and contents here in this notebook. The paper can be accessed here for reference: https://cdn.cboe.com/resources/vix/vixwhite.pdf  

# **Mechanics of the VIX**

### The VIX Index measures 30-day expected volatility of the S&P 500 Index. The components of the VIX Index are near- and next-term put and call options with more than 23 days and less than 37 days to expiration. These include SPX options with “standard” 3rd Friday expiration dates and “weekly” SPX options that expire every Friday, except the 3rd Friday of each month. Once each week, the SPX options used to calculate the VIX Index “roll” to new contract maturities. For example, on the second Tuesday in October, the VIX Index would be calculated using SPX options expiring 24 days later (i.e., “near- term”) and 31 days later (i.e., “next-term”). On the following day, the SPX options that expire in 30 calendar days would become the “near-term” options and SPX options that expire in 37 calendar days would be the “next-term” options.

### In this hypothetical example, the near-term options are “standard” SPX options with 25 days to expiration, the next-term options are P.M.-settled SPX Weeklys with 32 days to expiration; and the calculation reflects prices observed at 9:46 a.m. ET. For the purpose of calculating time to expiration, “standard” SPX options are deemed to expire at the open of trading on SPX settlement day - the third Friday of the month, and “weekly” SPX options are deemed to expire at the close of trading (i.e., 4:00 p.m. ET).

### The VIX calculation measures time to expiration, T, in calendar days and divides each day into minutes in order to replicate the precision that is commonly used by professional option and volatility traders. The time to expiration is given by the following expression:

### T = {$M_{Current\,day} + M_{Settlement\,day} + M_{Other\,days}$} / Minutes in a year

### Where:
### $M_{Current\,day}$ = minutes remaining until midnight of the current day
### $M_{Settlement\,day}$ = minutes from midnight until 9:30 a.m. ET for "standard" SPX expirations; or minutes from midnight until 4:00 p.m. ET for "weekly" SPX options
### $M_{Other\,days}$ = total minutes in the days between current day and expiration day

In [ ]:
#def opt_expir(td):
  #return near_term_exp_ch, next_term_exp_ch

td = datetime.datetime(2020, 10, 26, 9, 46)
td = pytz.timezone('US/Eastern').localize(td)
print( td.utcoffset() )

lb = (td + timedelta(days=24))
#lb = pytz.timezone('US/Eastern').localize( lb )
ub = (td + timedelta(days=37))
#ub = pytz.timezone('US/Eastern').localize( ub )
print(lb)
print(td)

#print( pd.date_range('2019-12-31','2020-12-31',freq='WOM-3FRI') )

#print( pd.date_range(lb, ub,freq='W-FRI', tz='US/Eastern') )
print( list( pd.date_range(lb, ub,freq='W-FRI', tz='US/Eastern').values ) )
fridays = list( pd.date_range(lb, ub,freq='W-FRI', tz='US/Eastern').values )
third_fri = list( pd.date_range(lb, ub,freq='WOM-3FRI', tz='US/Eastern').values )

print(fridays)
print(third_fri)
print( third_fri in fridays )
print('-----------------')
expir = []
for fri in fridays:
  if fri in third_fri: #Standard SPX
    print('standard')
    fri = pd.to_datetime(str(fri))
    #print(pytz.timezone('US/Eastern').localize( fri ) - td)
    print(fri.replace(hour=9, minute=30))
    print( pytz.timezone('US/Eastern').localize( fri.replace(hour=9, minute=30) ) - td )
    expir.append( pytz.timezone('US/Eastern').localize( fri.replace(hour=9, minute=30) ) )

  else:               #Weekly SPX
    print('weekly')
    fri = pd.to_datetime(str(fri))
    #print(pytz.timezone('US/Eastern').localize( fri ) - td)
    print(fri.replace(hour=16, minute=0))
    print( pytz.timezone('US/Eastern').localize( fri.replace(hour=16, minute=0) ) - td )
    expir.append( pytz.timezone('US/Eastern').localize( fri.replace(hour=16, minute=0) ) )

print('~~~~~~~~~~~~~~~~~~~~~~~~~')
print(expir)
print('~~~~~~~~~~~~~~~~~~~~~~~~~')
print('- - - - - - - - - - - - - - - - - - - ')
near_term_exp_ch = min(expir)
next_term_exp_ch = max(expir)

print('- - - - - - - - - - - - - - - - - - - ')
print(near_term_exp_ch.utcoffset() )
print(next_term_exp_ch.utcoffset() )

-1 day, 20:00:00
2020-11-19 09:46:00-04:00
2020-10-26 09:46:00-04:00
[numpy.datetime64('2020-11-20T13:46:00.000000000'), numpy.datetime64('2020-11-27T13:46:00.000000000')]
[numpy.datetime64('2020-11-20T13:46:00.000000000'), numpy.datetime64('2020-11-27T13:46:00.000000000')]
[numpy.datetime64('2020-11-20T13:46:00.000000000')]
True
-----------------
standard
2020-11-20 09:30:00
25 days 00:44:00
weekly
2020-11-27 16:00:00
32 days 07:14:00
~~~~~~~~~~~~~~~~~~~~~~~~~
[Timestamp('2020-11-20 09:30:00-0500', tz='US/Eastern'), Timestamp('2020-11-27 16:00:00-0500', tz='US/Eastern')]
~~~~~~~~~~~~~~~~~~~~~~~~~
- - - - - - - - - - - - - - - - - - - 
- - - - - - - - - - - - - - - - - - - 
-1 day, 19:00:00
-1 day, 19:00:00


### Using 9:46 a.m. ET as the time of the calculation, T for the near-term and next-term options, T1 and T2, respectively, is:

**$T_1$ = {854 + 510 + 34,560} / 525,600 = 0.0683486**

**$T_2$ = {854 + 900 + 44,640} / 525,600 = 0.0882686**  

In [ ]:
days_per_year = 365 #calendar, not trading days
minutes_per_year = days_per_year * 24 * 60
minutes_per_day = 24 * 60


def M_current_day(td):
  M_current_day = datetime.datetime.combine(td.date(), datetime.datetime.max.time()) #midnight of the current day
  M_current_day = pytz.timezone('US/Eastern').localize(M_current_day)
  M_current_day -= td
  M_current_day = round(M_current_day.seconds / 60)
  #print(M_current_day)
  return M_current_day

def M_settlement_day(settlement_date):
  ###
  settlement_date  = settlement_date.astimezone('America/Chicago')
  ###
  M_settlement_day = datetime.datetime.combine(settlement_date.date(), datetime.datetime.min.time()) #midnight of the current day
  M_settlement_day = pytz.timezone('America/Chicago').localize(M_settlement_day)
  M_settlement_day = settlement_date - M_settlement_day
  M_settlement_day = M_settlement_day.seconds / 60
  ###
  #M_settlement_day = M_settlement_day.astimezone('US/Eastern')
  ###
  #print(M_settlement_day)
  return M_settlement_day

def M_other_days(settlement_day, current_day):
  settlement_day = datetime.datetime.combine(settlement_day.date(), datetime.datetime.min.time())
  current_day = datetime.datetime.combine(current_day.date(), datetime.datetime.max.time())
  M_other_days = settlement_day - current_day
  M_other_days = M_other_days.days * (24*60)
  #print(M_other_days)
  return M_other_days

def opt_time_to_mat(M_current_day, M_settlement_day, M_other_days):
  days_per_year = 365
  minutes_per_year = days_per_year * 24 * 60
  T = M_current_day + M_settlement_day + M_other_days
  T /= minutes_per_year
  return T

T1 = opt_time_to_mat(M_current_day=M_current_day(td=td),
                     M_settlement_day=M_settlement_day(near_term_exp_ch),
                     M_other_days=M_other_days(settlement_day = near_term_exp_ch, current_day=td))

T2 = opt_time_to_mat(M_current_day=M_current_day(td=td),
                     M_settlement_day=M_settlement_day(next_term_exp_ch),
                     M_other_days=M_other_days(settlement_day = next_term_exp_ch, current_day=td))


print(f'Near-term time-to-maturity: {T1:.7f} years')
print(f'Next-term time-to-maturity: {T2:.7f} years')

Near-term time-to-maturity: 0.0683486 years
Next-term time-to-maturity: 0.0882686 years


### The risk-free interest rates, $R_1$ and $R_2$, are yields based on U.S. Treasury yield curve rates (commonly referred to as “Constant Maturity Treasury” rates or CMTs), to which a cubic spline is applied to derive yields on the expiration dates of relevant SPX options. As such, the VIX Index calculation may use different risk-free interest rates for near- and next-term options. In this example, assume that $R_1$ = 0.0305% for the near-term options and that $R_2$ = 0.0286% for the next-term options. Note in this example, $T_2$ uses a value of 900 for MSettlement day, which reflects the 4:00 p.m. ET expiration time of the next-term SPX Weeklys options. Since many of the interim calculations are repetitive, only representative samples appear below.

In [ ]:
R1 = 0.0305 / 100
R2 = 0.0286 / 100

# ***STEP 1*** - Select the options to be used in the VIX calculation

In [ ]:
"""
#I use google sheets to read in the spreadsheet -- you can use read_csv
gc = gspread.authorize(GoogleCredentials.get_application_default())
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1iCTSsGRwz8-ASJcCHEwKVSUXF-yPMnkruCUvPp3lunw/edit#gid=1712546072')

def opt_df(opt_maturity_str):
  sheet = wb.worksheet(opt_maturity_str)
  df = pd.DataFrame( sheet.get_all_records() )
  df.set_index('Strike', drop=True,inplace=True)
  return df

near_term_opts = opt_df('Near-Term Options')
next_term_opts = opt_df('Next-Term Options')
"""

In [3]:
near_term_opts=pd.read_excel('VIX_sample_data.xlsx',sheet_name='Near-Term Options')
next_term_opts=pd.read_excel('VIX_sample_data.xlsx',sheet_name='Next-Term Options')

### The selected options are out-of-the-money SPX calls and out-of-the-money SPX puts centered around an at-the-money strike price, $K_0$. Only SPX options quoted with non-zero bid prices are used in the VIX calculation.

### **One important note**: as volatility rises and falls, the strike price range of options with non- zero bids tends to expand and contract. As a result, the number of options used in the VIX calculation may vary from month-to-month, day-to-day and possibly, even minute-to- minute.

### For each contract month:

### a) Determine the forward SPX level, F, by identifying the strike price at which the absolute difference between the call and put prices is smallest. The call and put prices in the following table reflect the average of each option’s bid / ask quotation. As shown below, the difference between the call and put prices is smallest at the **1965** strike for the near- and the **1960** strike for the next-term options.

### Using the 1965 call and put in the near-term, and the 1960 call and put in the next-term contract applied to the formula:
### F = Strike Price + $e^{RT}$ × (Call Price – Put Price),

#### the forward index prices, F1 and F2, for the near- and next-term options, respectively, are:

In [ ]:
def fwd_price(df, R, T):
  df['Absolute Difference'] = df[['Call Bid', 'Call Ask']].mean(axis=1)
  df['Absolute Difference'] -= df[['Put Bid', 'Put Ask']].mean(axis=1)
  df['Absolute Difference'] = df['Absolute Difference'].abs()
  #min_diff = df['Absolute Difference'].idxmin()

  row_min_diff = df[ df['Absolute Difference'] == df['Absolute Difference'].min() ]
  Call = row_min_diff[['Call Bid', 'Call Ask']].mean(axis=1).values
  Put = row_min_diff[['Put Bid', 'Put Ask']].mean(axis=1).values
  F = float(  row_min_diff.index[0] + np.exp(R*T) * (Call - Put) )
  return F

F2 = fwd_price(next_term_opts, R2, T2)
F1 = fwd_price(near_term_opts, R1, T1)

### b) Next, determine **$K_0$** -- the strike price immediately below the forward index level, F -- for the near- and next-term options. In this example, **$K_{0,1}$** = **1960** and **$K_{0,2}$** = **1960**.

In [ ]:
def min_diff(df, F):
  min_diff = next_term_opts.index[next_term_opts.index < F2].max()
  return min_diff

min_diff_near = min_diff(near_term_opts, F1)
min_diff_next = min_diff(next_term_opts, F2)

### c) Select out-of-the-money put options with strike prices < $K_0$. Start with the put strike immediately lower than $K_0$ and move to successively lower strike prices. Exclude any put option that has a bid price equal to zero (i.e., no bid). As shown below, once two puts with consecutive strike prices are found to have zero bid prices, no puts with lower strikes are considered for inclusion. **(Note that the 1350 and 1355 put options are not included despite having non-zero bid prices.)**

### d) Next, select out-of-the-money call options with strike prices > $K_0$. Start with the call strike immediately higher than $K_0$ and move to successively higher strike prices, excluding call options that have a bid price of zero. As with the puts, once two consecutive call options are found to have zero bid prices, no calls with higher strikes are considered. (**Note that the 2225 call option is not included despite having a non- zero bid price.**)

In [ ]:
def cons_bids(col, threshold=2, opt_type='Put'):
  #Puts
  indicator = 0
  if opt_type == 'Put':
    for i in sorted(col.index, reverse=True):
      #print( col.loc[i] )
      #print( col.iloc[ col.index.get_loc(i)-1] )
      if (col.loc[i] == 0) & (col.iloc[ col.index.get_loc(i)-1] == 0):
        indicator = threshold
      if indicator == threshold:
        return col[ col.index > i][:,]
    if indicator < threshold:
      return col

  #Calls
  else:
    for i in sorted(col.index, reverse=False):
      if (col.loc[i] == 0) & (col.iloc[ col.index.get_loc(i)-1] == 0):
        indicator = threshold
      if indicator == threshold:
        return col[ col.index < i][:,]
    if indicator < threshold:
      return col

#col[ col.index < i][:-1,]

In [ ]:
K0_near = near_term_opts.index[near_term_opts.index < F1 ][-1]
K0_next = next_term_opts.index[next_term_opts.index < F2 ][-1]

#Near-Term Options
#Puts
near_puts_used = near_term_opts[ near_term_opts.index < K0_near]
near_puts_used = cons_bids(col=near_puts_used['Put Bid'], threshold=2, opt_type='Put')
near_puts_used = near_puts_used[near_puts_used>0]
#Calls
near_calls_used = near_term_opts[ near_term_opts.index > K0_near]
near_calls_used = cons_bids(col=near_calls_used['Call Bid'], threshold=2, opt_type='Call')
near_calls_used = near_calls_used[near_calls_used>0]

#Next-Term Options
#Puts
next_puts_used = next_term_opts[ next_term_opts.index < K0_next]
next_puts_used = cons_bids(col=next_puts_used['Put Bid'], threshold=2, opt_type='Put')
next_puts_used = next_puts_used[next_puts_used>0]
#Calls
next_calls_used = next_term_opts[ next_term_opts.index > K0_next]
next_calls_used = cons_bids(col=next_calls_used['Call Bid'], threshold=2, opt_type='Call')
next_calls_used = next_calls_used[next_calls_used>0]

### e) Finally, select **both** the put and call with strike price **$K_0$**. Notice that two options are selected at **$K_0$**, while a single option, either a put or a call, is used for every other strike price.)

### The following table contains the options used to calculate the VIX in this example. VIX uses the average of quoted bid and ask, or mid-quote, prices for each option selected. The $K_0$ put and call prices are averaged to produce a single value. The price used for the 920 strike in the near-term is, therefore, (24.25 + 21.30)/2 = 22.775; and the price used in the next-term is (27.30 + 24.90)/2 = 26.10.

In [ ]:
def opts_used(opt_df, calls_used, puts_used, min_diff):
  opts_used = opt_df.loc[calls_used.index.append(puts_used.index)]
  opts_used['Option Type'] = opts_used.index
  opts_used['Option Type'] = opts_used['Option Type'].apply( lambda x: 'Call' if x > min_diff else 'Put')
  opts_used['Mid-quote Price'] = 0

  opts_used['Call Midquote'] = opts_used[['Call Bid', 'Call Ask']].mean(axis=1).values
  opts_used['Put Midquote'] = opts_used[['Put Bid', 'Put Ask']].mean(axis=1).values

  opts_used['Mid-quote Price'] =  opts_used.apply(lambda x : x['Call Midquote'] if x['Option Type'] == 'Call' else x['Put Midquote'], axis=1)
  opts_used = opts_used[['Mid-quote Price', 'Option Type']]

  put_call_avg = opt_df.loc[min_diff][['Call Bid', 'Call Ask', 'Put Bid', 'Put Ask']].mean()
  new_row = pd.Series(data={'Mid-quote Price':put_call_avg, 'Option Type':'Put/Call Average'}, name=min_diff)
  opts_used = opts_used.append(new_row, ignore_index=False)

  opts_used.sort_index(inplace=True)
  return opts_used

near_opts_used = opts_used(near_term_opts, near_calls_used, near_puts_used, min_diff_near)
next_opts_used = opts_used(next_term_opts, next_calls_used, next_puts_used, min_diff_next)

# ***STEP 2*** - Calculate volatility for both near-term and next-term options

### Applying the VIX formula (1) to the near-term and next-term options with time to expiration of $T_1$ and $T_2$, respectively, yields:

### $\sigma^2_1 = \frac{2}{T_1} \sum_i \frac{\Delta K_i}{K_i^2}e^{RT_i} Q(K_i) - \frac{1}{T_1}[\frac{F_1}{K_0}-1]^2$

### $\sigma^2_2 = \frac{2}{T_2} \sum_i \frac{\Delta K_i}{K_i^2}e^{RT_2} Q(K_i) - \frac{1}{T_2}[\frac{F_2}{K_0}-1]^2$

### VIX is an amalgam of the information reflected in the prices of all of the selected options. The contribution of a single option to the VIX value is proportional to ∆K and the price of that option, and inversely proportional to the square of the option’s strike price.

### Generally, ∆$K_i$ is half the difference between the strike prices on either side of $K_i$. For example, the ∆K for the next-term 1325 Put is 37.5: $∆K_{1325 Put}$ = (1350 – 1275)/2. At the upper and lower edges of any given strip of options, ∆$K_i$ is simply the difference between $K_i$ and the adjacent strike price. In this example, the 1370 Put is the lowest strike in the strip of near-term options and 425 is the adjacent strike. Therefore, ∆$K_{1370 Put}$ = 5 (i.e., 1375 – 1370).

### The contribution of the near-term 400 Put is given by:

### $\frac{\Delta K_{1370 Put}}{K^2_{1370 Put}} e^{RT_1} Q(1370 Put) = \frac{5}{1370} e^{.000305(0.0683486)} (0.20) = 0.0000005328$


### A similar calculation is performed for each option. The resulting values for the near-term options are then summed and multiplied by 2/$T_1$ . Likewise, the resulting values for the next-term options are summed and multiplied by 2/$T_2$ . The table below summarizes the results for each strip of options.

In [ ]:
def strik_contr(opts_used, R, T):
  opts_used['Contribution by Strike'] = opts_used.index
  opts_used['Contribution by Strike'] = opts_used['Contribution by Strike'].shift(-1) - opts_used['Contribution by Strike'].shift(1)
  opts_used['Contribution by Strike'] /= 2
  idx_num = opts_used.columns.tolist().index('Contribution by Strike')
  opts_used.iloc[0, idx_num] = (opts_used.iloc[1].name - opts_used.iloc[0].name)
  opts_used.iloc[-1,idx_num] = (opts_used.iloc[-1].name - opts_used.iloc[-2].name)
  opts_used['Contribution by Strike'] /= opts_used.index**2
  opts_used['Contribution by Strike'] *= np.exp(R*T)
  opts_used['Contribution by Strike'] *= opts_used['Mid-quote Price']
  return opts_used

next_opts_used = strik_contr(next_opts_used, R2, T2)
near_opts_used = strik_contr(near_opts_used, R1, T1)

### Next, calculate $\frac{1}{T} [\frac{F}{K_0} - 1]^2$ for the near-term ($T_1$) and the next-term ($T_2$):

### $\frac{1}{T_1} [\frac{F_1}{K_0} - 1]^2 = \frac{1}{0.0683486} [\frac{1962.89996}{1960} - 1]^2$ = 0.00003203

### $\frac{1}{T_2} [\frac{F_2}{K_0} - 1]^2 = \frac{1}{0.0882686} [\frac{1962.40006}{1960} - 1]^2$ = 0.00001699

### Now calculate $\sigma_1^2$ and $\sigma_2^2$:

### $\sigma^2_1 = \frac{2}{T_1} \sum_i \frac{\Delta K_i}{K_i^2}e^{RT_i} Q(K_i) - \frac{1}{T_1}[\frac{F_1}{K_0}-1]^2$ = 0.018495 - 0.00003203 = **0.01846292**

### $\sigma^2_2 = \frac{2}{T_2} \sum_i \frac{\Delta K_i}{K_i^2}e^{RT_2} Q(K_i) - \frac{1}{T_2}[\frac{F_2}{K_0}-1]^2$ = 0.018838 - 0.00001699 = **0.01882101**

In [ ]:
def first_term(opts_used, T):
  first_term = opts_used['Contribution by Strike'].sum()
  first_term *= (2/T)
  return first_term

def second_term(F,K0,T):
  second_term = (F/K0)
  second_term -= 1
  second_term **= 2
  second_term *= (1/T)
  return second_term

sigma_square_near = first_term(near_opts_used, T1) - second_term(F1, K0_near, T1)
sigma_square_next = first_term(next_opts_used, T2) - second_term(F2, K0_next, T2)

### ***STEP 3*** - Calculate the 30-day weighted average of $\sigma^2_1$ and $\sigma^2_2$. Then take the square root of that value and multiply by 100 to get VIX index value

### VIX = 100 x $\sqrt{\{T_1\sigma_1^2 [\frac{N_{T_2} - N_{T_{30}} }{N_{T_2} - N_{T_1}}] + T_2\sigma_2^2 [\frac{N_{T_{30}} - N_{T_1} }{N_{T_2} - N_{T_1}}] \}x\frac{N_{365}}{N_{30}}}$

### When the near-term options have less than 30 days to expiration and the next-term options have more than 30 days to expiration, the resulting VIX value reflects an interpolation of $\sigma_1^2$ and $\sigma_2^2$ ;i.e., each individual weight is less than or equal to 1 and the sum of the weights equals 1.

### $N_{T1}$ = number of minutes to settlement of the near-term options (35,924)
### $N_{T2}$ = number of minutes to settlement of the next-term options (46,394)
### $N_{T30}$ = number of minutes in 30 days (30 × 1,440 = 43,200)
### $N_{T365}$ = number of minutes in a 365-day year (365 ×1,440 = 525,600)

In [ ]:
#minutes_per_year
minsutes_30_days = 60 * 24 * 30
N_T1 = T1 * 365 * 24 * 60
N_T2 = T2 * 365 * 24 * 60

vix = T1 * sigma_square_near * ( (N_T2 - minsutes_30_days) / (N_T2-N_T1) )
vix += T2 * sigma_square_next * ( (minsutes_30_days-N_T1) / (N_T2-N_T1) )
vix *= (minutes_per_year/minsutes_30_days)
vix = np.sqrt(vix)
vix *= 100
vix

13.685820537947876

### VIX = 100 x $\sqrt{\{0.0683486 x 0.0184629 x [\frac{46,394 - 43,200}{46,394 - 35,924}] + 0.0882686 x 0.018821 x [\frac{43,200 - 35,924}{43,394 - 35,924}] \}x\frac{525,600}{43,200} }$

### **VIX = 100 x 0.13685821 = 13.69**